# Vitessce Widget Tutorial

# Visualization of a SpatialData object

## Import dependencies


In [4]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
import zipfile
import shutil

from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)

from vitessce.data_utils import (
    sdata_morton_sort_points,
    sdata_points_process_columns,
    sdata_points_write_bounding_box_attrs,
    sdata_points_modify_row_group_size,
    sdata_morton_query_rect,
)

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [5]:
from spatialdata import read_zarr

In [6]:
data_dir = "data"
zip_filepath = join(data_dir, "xenium_rep1_io.spatialdata.zarr.zip")
spatialdata_filepath = join(data_dir, "xenium_rep1_io.spatialdata.zarr")

In [7]:
if not isdir(spatialdata_filepath):
    if not isfile(zip_filepath):
        os.makedirs(data_dir, exist_ok=True)
        urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/xenium_rep1_io.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(data_dir)
        os.rename(join(data_dir, "data.zarr"), spatialdata_filepath)
        
        # This Xenium dataset has an AnnData "raw" element.
        # Reference: https://github.com/giovp/spatialdata-sandbox/issues/55
        raw_dir = join(spatialdata_filepath, "tables", "table", "raw")
        if isdir(raw_dir):
            shutil.rmtree(raw_dir)

In [8]:
sdata = read_zarr(spatialdata_filepath)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/vitessce-python/docs/notebooks/data/xenium_rep1_io.spatialdata.zarr
├── Images
│     ├── 'morphology_focus': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 8) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (167780, 313)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), morphology_mip (Images), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes)

In [9]:
sdata["transcripts"].shape[0].compute()

42638083

In [10]:
sdata.tables["table"].X = sdata.tables["table"].X.toarray()
sdata.tables["dense_table"] = sdata.tables["table"]
sdata.write_element("dense_table")

In [11]:
# TODO: store the two separate images as a single image with two channels.
# Similar to https://github.com/EricMoerthVis/tissue-map-tools/pull/12

In [12]:
sdata.tables['table'].obs

,cell_id,transcript_counts,control_probe_counts,control_codeword_counts,total_counts,cell_area,nucleus_area,region
0,1,28,1,0,29,58.387031,26.642188,cell_circles
1,2,94,0,0,94,197.016719,42.130781,cell_circles
2,3,9,0,0,9,16.256250,12.688906,cell_circles
3,4,11,0,0,11,42.311406,10.069844,cell_circles
4,5,48,0,0,48,107.652500,37.479688,cell_circles
...,...,...,...,...,...,...,...,...
167775,167776,229,1,0,230,220.452813,60.599688,cell_circles
167776,167777,79,0,0,79,37.389375,25.242344,cell_circles
167777,167778,397,0,0,397,287.058281,86.700000,cell_circles
167778,167779,117,0,0,117,235.354375,25.197188,cell_circles


In [13]:
sdata

SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/vitessce-python/docs/notebooks/data/xenium_rep1_io.spatialdata.zarr
├── Images
│     ├── 'morphology_focus': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 8) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
└── Tables
      ├── 'dense_table': AnnData (167780, 313)
      └── 'table': AnnData (167780, 313)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), morphology_mip (Images), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes)

In [14]:
sdata.points['transcripts'].head()

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv
0,4.395842,328.666473,12.019493,SEC11C,565,0,281474976710656,18.662479
1,5.074415,236.964844,7.608510,NegControlCodeword_0502,540,0,281474976710657,18.634956
2,4.702023,322.797150,12.289083,SEC11C,562,0,281474976710658,18.662479
3,4.906601,581.428650,11.222615,DAPK3,271,0,281474976710659,20.821745
4,5.660699,720.851746,9.265523,TCIM,291,0,281474976710660,18.017488


## Sorting Points and creating a new Points element in the SpatialData object

In [15]:
sdata = sdata_morton_sort_points(sdata, "transcripts")

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/spatialdata/_core/_elements.py:115: UserWarning: Key `transcripts` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [16]:
sdata.points["transcripts"].attrs["spatialdata_attrs"].get("feature_key")

'feature_name'

In [17]:
# Add feature_index column to dataframe, and reorder columns so that feature_name (dict column) is the final column (all the way to the right)
# Convert feature_name column to feature_index column.
ddf = sdata_points_process_columns(sdata, "transcripts", var_name_col="feature_name", table_name="table")

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/dask/dataframe/core.py:4448: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('feature_name', 'category'))

  warnings.warn(meta_warning(meta))


In [18]:
ddf.head()

,x,y,z,cell_id,overlaps_nucleus,transcript_id,qv,x_uint,y_uint,morton_code_2d,feature_name_codes,feature_name
677718,16.058575,17.155981,6.504758,-1,0,281474977398719,40.000000,156,152,50128,113,ERBB2
542861,49.912006,13.121742,6.657761,-1,0,281474977261746,20.909292,451,104,96389,294,TOMM7
817519,50.067265,14.154890,6.614446,-1,0,281474977540499,40.000000,452,116,96816,256,SERHL2
600252,40.367912,24.953489,6.542048,-1,0,281474977320069,40.000000,367,246,114301,250,SCD
518878,56.408573,16.603086,6.755222,-1,0,281474977237434,20.625040,507,146,120653,296,TPD52


In [19]:
sdata["transcripts_with_morton_codes"] = ddf
sdata.write_element("transcripts_with_morton_codes")

In [20]:
sdata_points_write_bounding_box_attrs(sdata, "transcripts_with_morton_codes")

In [21]:
sdata_points_modify_row_group_size(sdata, "transcripts_with_morton_codes", row_group_size=25_000)

In [22]:
# Done

In [23]:
import pyarrow.parquet as pq
from os.path import join

# Check the number of row groups
parquet_file = pq.ParquetFile(join(sdata.path, "points", "transcripts_with_morton_codes", "points.parquet", "part.0.parquet"))

# Get the total number of row groups
num_groups = parquet_file.num_row_groups
num_groups

92

## Configure Vitessce

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.18",
    name='MERFISH SpatialData Demo',
)
# Add data to the configuration:
wrapper = SpatialDataWrapper(
    sdata_path=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    image_path="images/rasterized",
    table_path="tables/table",
    obs_feature_matrix_path="tables/table/X",
    obs_spots_path="shapes/cells",
    coordinate_system="global",
    coordination_values={
        # The following tells Vitessce to consider each observation as a "spot"
        "obsType": "cell",
    }
)
dataset = vc.add_dataset(name='MERFISH').add_object(wrapper)

# Add views (visualizations) to the configuration:
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view("featureList", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
obs_sets = vc.add_view("obsSets", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    'spotLayer': CL([{
        'obsType': 'cell',
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSpots"))

vc.link_views([spatial, layer_controller, feature_list, obs_sets], ['obsType'], [wrapper.obs_type_label])

# Layout the views
vc.layout(spatial | (feature_list / layer_controller / obs_sets));

### Render the widget

In [ ]:
vw = vc.widget()
vw